In [7]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS



In [8]:
import os
from constants import openai_key

os.environ["OPENAI_API_KEY"]=openai_key


In [9]:
pdfreader=PdfReader('PMFBY_Guidelines.pdf')

In [10]:
from typing_extensions import Concatenate

raw_text=''

for i,page in enumerate(pdfreader.pages):
    content=page.extract_text()
    if content:
        raw_text+=content

In [12]:
text_splitter=CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
texts=text_splitter.split_text(raw_text)

In [13]:
len(texts)

448

In [14]:
embeddings=OpenAIEmbeddings()

In [15]:
document_search=FAISS.from_texts(texts,embeddings)

In [16]:
document_search

In [17]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI



In [18]:
chain=load_qa_chain(OpenAI(),chain_type="stuff")

In [21]:
query="within how many days such the yield loss be calculated?"
docs=document_search.similarity_search(query)
chain.run(input_documents=docs,question=query)

' The yield loss should be calculated within 15 days from the occurrence of the adverse seasonal event.'